In [1]:
import pandas as pd
import datacompy
import gc
# Ouali benchmark file only includes auto aligned data
benchmark_file = 'input/benchmark-nomsATC.tsv'
benchmark_undefined_file = 'input/benchmark-undefined-nomsATC.tsv'
rero_files = ['input/RERO-aligned-2021-11.txt', 'input/RERO-google-extract-20220330.tsv']
rero_concord_file = 'input/20220228_concordances_id_rnv_id_rero_4_headings.csv'

abes_files = ['input/abes-alignements-surs.csv']
abes_noalign_file = 'input/abes-nonalign-only.csv'

set_name = 'nomsATC'

output_folder = 'output'

# Main names for various export files
common_file = 'communs'
common_noalign_file = 'communs-nonalignements'
divergences_file = 'divergences'
div_noalign_file = 'divergences-nonalignements'
missed_file = 'manquants'

### Préparation des données Ouali

Commençons par charger les données exportées depuis Ouali en les chargeant dans des dataframes.

Pour tous les chargements de données, on s'assure que Python traite toutes les valeurs comme texte, sinon il va convertir certaines colonnes contenant des identifiants en nombres, ce qui peut poser problème par la suite si on les compare aux mêmes données chargées comme texte ou si on essaie de faire une opération join sur ces colonnes. Pour cela, on spécifie le paramètre `dtype`.

In [2]:
ouali_data = pd.read_csv(benchmark_file, sep='\t', dtype = str)
ouali_undefined_data = pd.read_csv(benchmark_undefined_file, sep='\t', dtype = str)
# Create origin and target columns that are named the same as in the RERO files, for ease of merging
ouali_data['source'] = ouali_data['id source']
ouali_undefined_data['source'] = ouali_undefined_data['id source']
ouali_data['cible'] = ouali_data['id cible']
# .copy() is necessary to avoid SettingWithCopyWarning when merging
ouali_align = ouali_data.query('`nombre de candidats` != "0" & `décision d\'alignement` == "auto"').copy()
ouali_no_align = ouali_data.query('`nombre de candidats` == "0" & `décision d\'alignement` == "auto"').copy()

print('Alignements Ouali: ' + str(len(ouali_align)))
print('Non-alignements Ouali: ' + str(len(ouali_no_align)))
print('Alignements à arbitrer (pas de décision): ' + str(len(ouali_undefined_data)))

Alignements Ouali: 641618
Non-alignements Ouali: 352076
Alignements à arbitrer (pas de décision): 170975


### Préparation des données RERO

Pour la comparaison avec RERO, commençons par charger ces données également dans des dataframes.

In [3]:
rero_data = pd.concat((pd.read_csv(f, sep='\t', encoding = "ISO-8859-1", dtype = str) for f in rero_files))
rero_data.columns =[column.replace(" ", "_") for column in rero_data.columns]
# Remove any duplicate entries between the RERO extracts
rero_data.drop_duplicates(subset=['rero_id'],inplace=True)
rero_concord = pd.read_csv(rero_concord_file, dtype = str)
print("Nombre de concordances validées dans le fichier RERO: ",len(rero_data))

Nombre de concordances validées dans le fichier RERO:  25511


L'identifiant utilisé par RERO n'est pas le même que dans les exports Ouali. Il faut donc commencer par ajouter l'identifiant RNV aux exports RERO pour pouvoir comparer ces alignements à ceux exportés par Ouali.

*Note au sujet de l'affichage d'exemples:*
L'usage de la fonction `query` ne fonctionne pas très bien pour extraire quelques exemples de ces données, peut-être à cause des espaces. C'est pourquoi on utilise `loc` et `contains`. Enfn on spécifie `na=False` car ces colonnes contiennent des entrées `NaN` qu'il faut convertir en `False` pour pouvoir utiliser `loc`.

In [4]:
display(rero_data.loc[rero_data['author_heading'].str.contains("Dietrich, Andreas", na=False)])
display(rero_concord.loc[rero_concord['main_form'].str.contains("Dietrich, Andreas", na=False)])

,rero_id,idref_id,author_heading,subject_heading
4375,A003180228,257367934,"Dietrich, Andreas",NaN


,id,id_rnv,id_rero,id_rero_a,main_form,repo_id
124357,981023317235702851,(RNV_A)0000071017,(RERO)vtls003180228,NaN,"Dietrich, Andreas",rnv-nz-auth-atc
761184,981023380231902851,(RNV_A)0001220989,(RERO)vtls014190124,NaN,"Dietrich, Andreas, 1976-",rnv-nz-auth-atc


Il faut aligner les chiffres dans la colonne `rero_id` du dataframe `rero_data` avec soit ceux trouvés dans `id_rero` ou `id_rero_a` dans `rero_concord`. Pour toutes ces colonnes, il faut tout d'abord enlever les préfixes avant de pouvoir les utiliser comme point de comparaison.

In [5]:
rero_data['id_rero_join'] = rero_data.rero_id.str.extract('(\d+)')
rero_concord['id_rero_join'] = rero_concord['id_rero'].fillna(rero_concord['id_rero_a'])
rero_concord['id_rero_join'] = rero_concord.id_rero_join.str.extract('(\d+)')
display(rero_data.query('id_rero_join == "003180228"'))
display(rero_concord.query('id_rero_join == "003180228"'))

,rero_id,idref_id,author_heading,subject_heading,id_rero_join
4375,A003180228,257367934,"Dietrich, Andreas",NaN,003180228


,id,id_rnv,id_rero,id_rero_a,main_form,repo_id,id_rero_join
124357,981023317235702851,(RNV_A)0000071017,(RERO)vtls003180228,NaN,"Dietrich, Andreas",rnv-nz-auth-atc,003180228


Maintenant que c'est fait, on peut fusionner les deux tables et ainsi obtenir notre fichier de comparaison. On renomme également les colonnes `id` et  `idref_id` en `source` et `cible` respectivement, qui sont les termes utilisés dans l'export Ouali pour pouvoir les comparer à ce dernier.

In [6]:
rero_aligns = pd.merge(rero_data, rero_concord, on='id_rero_join', how="inner")
rero_aligns['source'] = rero_aligns['id']
rero_aligns['cible'] = rero_aligns['idref_id']
display(rero_aligns.query('id_rero_join == "003180228"'))
print('Nombre de concordiances RERO-IdRef validées: ' + str(len(rero_aligns)))

,rero_id,idref_id,author_heading,subject_heading,id_rero_join,id,id_rnv,id_rero,id_rero_a,main_form,repo_id,source,cible
4331,A003180228,257367934,"Dietrich, Andreas",NaN,003180228,981023317235702851,(RNV_A)0000071017,(RERO)vtls003180228,NaN,"Dietrich, Andreas",rnv-nz-auth-atc,981023317235702851,257367934


Nombre de concordiances RERO-IdRef validées: 20372


Nous avons maintenant un fichier prêt à être utilisé pour la comparaison avec Ouali.

### Comparaison des données Ouali avec RERO

In [7]:
common_source_rero = pd.merge(rero_aligns, ouali_align, how="inner", on='source', suffixes=("_rero", "_ouali"), copy=True)
print("Alignements Ouali présents dans fichier RERO: ", len(common_source_rero))

Alignements Ouali présents dans fichier RERO:  4858


In [8]:
outfile = output_folder + '/' + common_file + '-RERO-' + set_name + '.csv'
common_target_rero = pd.merge(rero_aligns, ouali_align, how="inner", on=['source', 'cible'], suffixes=("_rero", "_ouali"), copy=True)
common_target_rero.to_csv(outfile,columns=['source','cible','forme principale cible','main_form'],encoding="UTF-8",index=False)
print("Alignements Ouali validés par RERO: ", len(common_target_rero))
print("Pourcentage validé: ", round(len(common_target_rero)/len(common_source_rero)*100,1), "%")
print("Exportés dans ", outfile)

Alignements Ouali validés par RERO:  4699
Pourcentage validé:  96.7 %
Exportés dans  output/communs-RERO-nomsATC.csv


#### Différences entre Ouali et RERO

In [9]:
outfile = output_folder + '/' + divergences_file + '-RERO-' + set_name + '.csv'
divergences_rero = common_source_rero.loc[~(common_source_rero['cible_rero'] == common_source_rero['cible_ouali'])]
display(divergences_rero[['source','cible_ouali','cible_rero','forme principale cible','main_form']])
divergences_rero.to_csv(outfile,columns=['source','cible_ouali','cible_rero','forme principale cible','main_form'],encoding="UTF-8",index=False)
print("Alignements Ouali qui diffèrent dans RERO: ", len(divergences_rero))
print("Pourcentage non-validé: ", round(len(divergences_rero)/len(common_source_rero)*100,1), "%")
print("Exportés dans ", outfile)

,source,cible_ouali,cible_rero,forme principale cible,main_form
0,981023292733602851,098879456,117789259,"Abū Zahraẗ, Muḥammad","Abū Zahrah, Muḥammad, 1898-1974"
13,981023285513602851,137125097,030384109,Albert von Augsburg,"Albertus, De Saxonia"
21,981023291275002851,070455317,032474423,Ammonios Saccas 0175?-0242?,"Ammonius, Hermeiou"
22,981023296833702851,152792961,026685493,Anastase le Sinaïte saint 05..-0599,"Anastasius, Sinaita"
29,981023285003102851,030945054,167540831,Antonin saint 1389-1459,"Antonino, archevêque de Florence, saint"
...,...,...,...,...,...
4789,981023369854902851,258939575,258939575,"Boss, Sabine 1966-....","Boss, Sabine"
4816,981023298851002851,253657199,257106049,"Pury, Charles Albert de 1752-1833","Pury, Charles Albert de, 1713-1790"
4829,981023304204802851,076453650,259846554,"Feller, Christian 1943-....","Feller, Christian, expert-comptable"
4831,981023287378202851,150125666,068552963,"Bouyer, A.-C.","Courcier, veuve de Louis, fl. 1811-1821"


Alignements Ouali qui diffèrent dans RERO:  159
Pourcentage non-validé:  3.3 %
Exportés dans  output/divergences-RERO-nomsATC.csv


In [10]:
outfile = output_folder + '/' + div_noalign_file + '-RERO-' + set_name + '.csv'
div_nonalign_rero = pd.merge(rero_aligns, ouali_no_align, how="inner", on='source', suffixes=("_rero", "_ouali"), copy=True)
display(div_nonalign_rero[['source','cible_ouali','cible_rero','forme principale cible','main_form']])
div_nonalign_rero.to_csv(outfile,columns=['source','cible_ouali','cible_rero','forme principale cible','main_form'],encoding="UTF-8",index=False)
print("Non-alignements Ouali erronés: ", len(div_nonalign_rero))
print("Exportés dans ", outfile)

,source,cible_ouali,cible_rero,forme principale cible,main_form
0,981023302389702851,NaN,256648107,NaN,"Corrêa, Ruth Valadares"
1,981023303730702851,NaN,115972463,NaN,"Davydov, I︠U︡riĭ Nikolaevich"
2,981023300411502851,NaN,03394265X,NaN,"Derzhavin, Gavrila Romanovich"
3,981023291573902851,NaN,032211414,NaN,"Font Quer, P"
4,981023289136502851,NaN,03478375X,NaN,"Ganshina, Klavdii︠a︡ Aleksandrovna"
...,...,...,...,...,...
177,981023365104302851,NaN,261383205,NaN,"Eggendorffer, Aloïs"
178,981023357333702851,NaN,261383701,NaN,"Eggendorffer, Aloïs-Wilhelm"
179,981023348417402851,NaN,261219642,NaN,"Staeger, Dieter"
180,981023357657102851,NaN,261383205,NaN,"Eggendorffer, Louis"


Non-alignements Ouali erronés:  182
Exportés dans  output/divergences-nonalignements-RERO-nomsATC.csv


In [11]:
outfile = output_folder + '/' + missed_file + '-RERO-' + set_name + '.csv'
missed_rero = pd.merge(rero_aligns, ouali_undefined_data, how="inner", on='source', suffixes=("_rero", "_ouali"), copy=True)
display(missed_rero[['source','cible','main_form']])
missed_rero.to_csv(outfile,columns=['source','cible','main_form'],encoding="UTF-8",index=False)
print("Alignements manqués: ", len(missed_rero))
print("Exportés dans ", outfile)

,source,cible,main_form
0,981023281030102851,088915018,Ab'aigre
1,981023292513202851,057770697,"Jābirī, Muḥammad ʿĀbid"
2,981023281032102851,02664777X,"Abélard, Pierre"
3,981023291313702851,029184371,"Abū al-Qāsim Khalaf ibn ʿAbbās al-Zahrāwī, ca...."
4,981023300538902851,027477096,"Adam, de la Halle"
...,...,...,...
5079,981023321864402851,260779857,"Fleury, Robert"
5080,981023300160502851,260860387,"Maurer, Hans, 1865-1917"
5081,981023323570802851,026904195,"Gros, Christian"
5082,981023344002302851,261374540,"Pont, Jeanne"


Alignements manqués:  5084
Exportés dans  output/manquants-RERO-nomsATC.csv


Pour finir, faisons un peu d'ordre en effacant de la mémoire les dataframes utilisés pour la comparaison avec RERO avant de passer aux données ABES.

In [12]:
del rero_data, rero_concord, rero_aligns, common_source_rero, common_target_rero, divergences_rero, div_nonalign_rero, missed_rero
gc.collect()

73

### Préparation des données ABES

In [13]:
abes_align = pd.concat((pd.read_csv(f, dtype = str) for f in abes_files))
abes_noalign = pd.read_csv(abes_noalign_file, dtype = str)
print("Nombre de concordances validées dans le fichier ABES: ",len(abes_align))
print("Nombre de non-alignements validés dans le fichier ABES: ",len(abes_noalign))

Nombre de concordances validées dans le fichier ABES:  517622
Nombre de non-alignements validés dans le fichier ABES:  380473


### Comparaison des données Ouali avec l'ABES

Dans les données ABES, l'identifiant RNV est au milieu de la chaîne de caractères présents dans la colonne `ID_EC`. Il faut donc extraire ces chiffres et les placer dans une nouvelle colonne `source` pour pouvoir les comparer à Ouali. On renomme également la colonne `IdRef` en `cible` pour faciliter la comparaison. On fait la même chose pour le fichier des non-alignements.

In [14]:
abes_align['source']=abes_align['ID_EC'].str.extract(r'(\d{18})')
abes_align['cible']=abes_align['IdRef']
display(abes_align)
abes_noalign['source']=abes_noalign['ID_EC'].str.extract(r'(\d{18})')

,ID_EC,NB_RESU,IdRef,NOM,PRENOM,NOMCANDIDAT,PRENOMCANDIDAT,LST_HEUR,source,cible
0,renouvaud_ec_981023132827002851_bentoumi_farid,2,195225147,Bentoumi,Farid,Bentoumi,Farid,"cocontrib,titre",981023132827002851,195225147
1,renouvaud_ec_981023132827302851_daniel_marie,79,134948718,Daniel,Marie,Daniel,Marie,"cocontrib,titre",981023132827302851,134948718
2,renouvaud_ec_981023132828002851_boecker_virginia,1,251579700,Boecker,Virginia,Boecker,Virginia,"cocontrib,unica,titre",981023132828002851,251579700
3,renouvaud_ec_981023132829502851_gonzález_ramo...,57,19625714X,González,Ramón Gilberto‏,González,R. Gilberto,"cocontrib,titre",981023132829502851,19625714X
4,renouvaud_ec_981023132829802851_nadji_mehrdad,1,119903830,Nadji,Mehrdad,Nadji,Mehrdad,"cocontrib,titre",981023132829802851,119903830
...,...,...,...,...,...,...,...,...,...,...
517617,renouvaud_ec_981023880830202851_burch_maryr,4,101668813,Burch,Mary R,Burch,Mary R.,"cocontrib,titre,dewey",981023880830202851,101668813
517618,renouvaud_ec_981023880830302851_bailey_jons,41,101666810,Bailey,Jon S,Bailey,Jon S.,"cocontrib,titre,dewey",981023880830302851,101666810
517619,renouvaud_ec_981023880931602851_just_thomas,5,234740663,Just,Thomas,Just,Thomas,"cocontrib,titre",981023880931602851,234740663
517620,renouvaud_ec_981023880934802851_vallat_francis,23,168782219,Vallat,Francis,Vallat,Francis,"cocontrib,titre,dewey",981023880934802851,168782219


In [15]:
common_source_abes = pd.merge(abes_align, ouali_align, how="inner", on='source', suffixes=("_abes", "_ouali"), copy=True)
common_source_noalign_abes = pd.merge(abes_noalign, ouali_no_align, how="inner", on='source', suffixes=("_abes", "_ouali"), copy=True)
print("Alignements Ouali présents dans fichier ABES: ", len(common_source_abes))

Alignements Ouali présents dans fichier ABES:  428625


In [16]:
outfile = output_folder + '/' + common_file + '-ABES-' + set_name + '.csv'
common_target_abes = pd.merge(abes_align, ouali_align, how="inner", on=['source', 'cible'], suffixes=("_abes", "_ouali"), copy=True)
common_target_abes.to_csv(outfile,columns=['source','cible','forme principale cible','NOMCANDIDAT', 'PRENOMCANDIDAT'],encoding="UTF-8",index=False)
print("Alignements Ouali validés par l'ABES: ", len(common_target_abes))
print("Pourcentage validé: ", round(len(common_target_abes)/len(common_source_abes)*100,1), "%")
print("Exportés dans ", outfile)

Alignements Ouali validés par l'ABES:  425777
Pourcentage validé:  99.3 %
Exportés dans  output/communs-ABES-nomsATC.csv


In [17]:
outfile = output_folder + '/' + common_noalign_file + '-ABES-' + set_name + '.csv'
common_source_noalign_abes.to_csv(outfile,columns=['source'],encoding="UTF-8",index=False)
print("Non-alignements Ouali: ", len(ouali_no_align))
print("Non-alignements Ouali communs avec l'ABES: ", len(common_source_noalign_abes))
print("Pourcentage validé: ", round(len(common_source_noalign_abes)/len(ouali_no_align)*100,1), "%")
print("Exportés dans ", outfile)

Non-alignements Ouali:  352076
Non-alignements Ouali communs avec l'ABES:  251580
Pourcentage validé:  71.5 %
Exportés dans  output/communs-nonalignements-ABES-nomsATC.csv


#### Différences entre Ouali et l'ABES

In [18]:
outfile = output_folder + '/' + divergences_file + '-ABES-' + set_name + '.csv'
# Alignements pour lesquels Ouali et l'ABES ont pris une décision différente (cible différente pour une même source)
divergences_abes = common_source_abes.loc[~(common_source_abes['cible_abes'] == common_source_abes['cible_ouali'])]
display(divergences_abes[['source','cible_ouali','cible_abes','forme principale cible','NOMCANDIDAT', 'PRENOMCANDIDAT']])

# Notices qu'Ouali a alignées, mais l'ABES a pris une décision de non-alignement
divergences_NOT_abes = pd.merge(abes_noalign, ouali_align, how="inner", on=['source'], suffixes=("_abes", "_ouali"), copy=True)
divergences_NOT_abes['cible_ouali'] = divergences_NOT_abes['cible']
divergences_NOT_abes.insert(0,'cible_abes', "NaN")
display(divergences_NOT_abes[['source','cible_ouali','forme principale cible','NOMCANDIDAT', 'PRENOMCANDIDAT']])

divergences_abes.to_csv(outfile,columns=['source','cible_ouali','cible_abes','forme principale cible','NOMCANDIDAT', 'PRENOMCANDIDAT'],encoding="UTF-8",index=False)
# On ajoute à ce même fichier les cas où l'ABES a pris une décision de non-alignement (mode 'a' - append)
divergences_NOT_abes.to_csv(outfile,columns=['source','cible_ouali','cible_abes','forme principale cible','NOMCANDIDAT', 'PRENOMCANDIDAT'],encoding="UTF-8",index=False, mode='a')


print("Alignements Ouali qui diffèrent dans l'ABES: ", len(divergences_abes))
print("Pourcentage avec alignement différent: ", round(len(divergences_abes)/len(common_source_abes)*100,1), "%")
print("Alignements Ouali qui sont non-alignés par l'ABES: ", len(divergences_NOT_abes))
print("Pourcentage avec décision d'alignement différente: ", round(len(divergences_NOT_abes)/len(ouali_align)*100,1), "%")
print("Exportés dans ", outfile)

,source,cible_ouali,cible_abes,forme principale cible,NOMCANDIDAT,PRENOMCANDIDAT
0,981023132827302851,079574076,134948718,"Lino Daniel, Marie 1975-....",Daniel,Marie
26,981023265132802851,093513305,260039209,"Martín, Inés M.",Martín Rodrigo,Inés
41,981023280468002851,030342368,090411366,"Ramadier, Jacques O. 1920-1994",Ramadier,Jacques-Alexandre
70,981023280646902851,256376123,026896567,"Gómez Molina, Ramon 1931-....",Gómez de la Serna,Ramón
73,981023280671002851,128467924,244371164,"Philippe, Louis",Jobet,P. E. G
...,...,...,...,...,...,...
427674,981023769233802851,08335560X,07657783X,"Grant, Lester D. 19..-....",Grant,Lester Howard
427762,981023774429402851,254451012,23351760X,"Priolo, Giorgio Calogero 19..-....",Priolo,Calogero Giorgio
428482,981023878329802851,083907599,058504605,"Bresson, Solange 1958-....",Hadni-Bresson,Solange
428577,981023878916602851,034410104,257172912,"Teixeira, Pedro 1570?-1610?",Teixeira,Antony


,source,cible_ouali,forme principale cible,NOMCANDIDAT,PRENOMCANDIDAT
0,981023132837302851,101324170,"Halstead, Judith A.",NaN,NaN
1,981023132837902851,117218626,"Fulton, Janet S.",NaN,NaN
2,981023265125302851,17078763X,"Sharma, Avinash",NaN,NaN
3,981023265133302851,117619876,"Sánchez González, Juana María",NaN,NaN
4,981023280142402851,195165039,"Sāmbaśivaśāstri, Ke.",NaN,NaN
...,...,...,...,...,...
70130,981023880526202851,180993445,"Rossi, María Julia",NaN,NaN
70131,981023880629202851,070666083,"Kim, In-Seok 1950-....",NaN,NaN
70132,981023880630302851,174772556,"Poggi Repetto, Elvira",NaN,NaN
70133,981023880720002851,103686509,Malou,NaN,NaN


Alignements Ouali qui diffèrent dans l'ABES:  2848
Pourcentage avec alignement différent:  0.7 %
Alignements Ouali qui sont non-alignés par l'ABES:  70135
Pourcentage avec décision d'alignement différente:  10.9 %
Exportés dans  output/divergences-ABES-nomsATC.csv


In [28]:
outfile = output_folder + '/' + div_noalign_file + '-ABES-' + set_name + '.csv'

# Non-alignements Ouali pour lesquels l'ABES a trouvé un alignement
div_nonalign_abes = pd.merge(abes_align, ouali_no_align, how="inner", on='source', suffixes=("_abes", "_ouali"), copy=True)
display(div_nonalign_abes[['source','cible_ouali','cible_abes','forme principale cible','NOMCANDIDAT', 'PRENOMCANDIDAT']])

div_nonalign_abes.to_csv(outfile,columns=['source','cible_ouali','cible_abes','forme principale cible','NOMCANDIDAT', 'PRENOMCANDIDAT'],encoding="UTF-8",index=False)

print("Non-alignements Ouali erronés: ", len(div_nonalign_abes))
print("Pourcentage de divergence: ", round(len(div_nonalign_abes)/len(ouali_no_align)*100,1), "%")

print("Exportés dans ", outfile)


,source,cible_ouali,cible_abes,forme principale cible,NOMCANDIDAT,PRENOMCANDIDAT
0,981023132829502851,NaN,19625714X,NaN,González,R. Gilberto
1,981023280242602851,NaN,029602335,NaN,Berg,Johannes van den
2,981023280339102851,NaN,253746671,NaN,Santos,Domingos
3,981023280369802851,NaN,257014632,NaN,Rommey,Martin
4,981023280398402851,NaN,223370819,NaN,Albuquerque,Cristina
...,...,...,...,...,...,...
22815,981023878811702851,NaN,177222670,NaN,Mazzocco,Fabien
22816,981023878823702851,NaN,256786070,NaN,Thadeusz,Frank
22817,981023878827702851,NaN,257792090,NaN,Volckmer,Katharina
22818,981023878828502851,NaN,256979804,NaN,Sundance,Robert


Non-alignements Ouali erronés:  22820
Pourcentage de divergence:  6.5 %
Exportés dans  output/divergences-nonalignements-ABES-nomsATC.csv


In [30]:
outfile = output_folder + '/' + missed_file + '-ABES-' + set_name + '.csv'

# Alignements manqués
missed_abes = pd.merge(abes_align, ouali_undefined_data, how="inner", on='source', suffixes=("_abes", "_ouali"), copy=True)
display(missed_abes[['source','cible','NOMCANDIDAT','PRENOMCANDIDAT']])
missed_abes.to_csv(outfile,columns=['source','cible','NOMCANDIDAT','PRENOMCANDIDAT'],encoding="UTF-8",index=False)

# Non-alignements manqués
missed_noalign_abes = pd.merge(abes_noalign, ouali_undefined_data, how="inner", on='source', suffixes=("_abes", "_ouali"), copy=True)
missed_noalign_abes.insert(0,'cible', "NaN")
display(missed_noalign_abes[['source','cible','NOMCANDIDAT','PRENOMCANDIDAT']])
missed_noalign_abes.to_csv(outfile,columns=['source','cible','NOMCANDIDAT','PRENOMCANDIDAT'],encoding="UTF-8",index=False,mode='a')
print("Alignements manqués: ", len(missed_abes))
print("Non-alignements manqués: ", len(missed_noalign_abes))
print("Exportés dans ", outfile)

,source,cible,NOMCANDIDAT,PRENOMCANDIDAT
0,981023132827002851,195225147,Bentoumi,Farid
1,981023265033002851,136319254,Evangelista,Stefano-Maria
2,981023265034602851,034441956,Davies,Oliver
3,981023265123302851,026795248,Collins,Dennis
4,981023265125902851,031093892,Bank,Jan
...,...,...,...,...
66172,981023879035302851,220609551,Jeannin,Arnaud
66173,981023880630402851,166249092,Henderson,R. A.
66174,981023880931602851,234740663,Just,Thomas
66175,981023880934802851,168782219,Vallat,Francis


,source,cible,NOMCANDIDAT,PRENOMCANDIDAT
0,981023132829002851,NaN,NaN,NaN
1,981023132830402851,NaN,NaN,NaN
2,981023132835902851,NaN,NaN,NaN
3,981023265030302851,NaN,NaN,NaN
4,981023265128002851,NaN,NaN,NaN
...,...,...,...,...
58753,981023879026702851,NaN,NaN,NaN
58754,981023880526102851,NaN,NaN,NaN
58755,981023880826802851,NaN,NaN,NaN
58756,981023880834802851,NaN,NaN,NaN


Alignements manqués:  66177
Non-alignements manqués:  58758
Exportés dans  output/manquants-ABES-nomsATC.csv
